In [1]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,split
from pyspark.sql.types import IntegerType,StringType
import json

In [2]:
KAFKA_TOPIC = "lastStream"
KAFKA_SERVER = "localhost:9092"
spark = SparkSession \
    .builder \
    .appName("Streaming YouTubeLive") \
    .getOrCreate()

In [3]:
consumer=KafkaConsumer(KAFKA_TOPIC,bootstrap_servers=['localhost:9092'],auto_offset_reset='latest',api_version=(0,10))  # read data from kafka

In [4]:
hostName='localhost'
port=3306
database='streamdb'
table='youtubelive'
url=f'jdbc:mysql://{hostName}:{port}/{database}'

In [5]:
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [6]:
# try:
#     df = spark.write.jdbc(url=url, table=table, properties=properties)
# except Exception as e:
#     print(f'Error : {str(e)}')

In [ ]:
data=[]

counter=0

for msg in consumer:

    # data.append(msg.value)
    # print(msg.value)                                                                      
    value = msg.value.decode('utf-8') if isinstance(msg.value, bytes) else msg.value  # decode data
    splited_values=value.split(',')
    data.append(tuple(splited_values))
    # print(keys)

    if counter== 10: 
        # Taking data as string and spliting it into columns
        # df.printSchema()
        columns = [
            'Video_ID',
            'Channel_Name',
            'Country',
            'Category',
            'Comments',
            'Subscribers',
            'views_video',
            'likes',
            'date_live'       
           ]
        df = spark.createDataFrame(data,schema=columns)

        # dfColumns=df.withColumn('spiltedColumns',split(df["value"],","))
        # dataLive=dfColumns.selectExpr(
        # 'spiltedColumns[0] as Video_ID',
        # 'spiltedColumns[1] as Channel_Name',
        # 'spiltedColumns[2] as Country',
        # 'spiltedColumns[3] as Category',
        # 'cast(spiltedColumns[4] as int) Comments',
        # 'cast(spiltedColumns[5] as int) as Subscribers',
        # 'cast(spiltedColumns[6] as int) as views_video',
        # 'cast(spiltedColumns[7] as int)as likes',
        # 'to_date(spiltedColumns[8], "yyyy-MM-dd") as date_live'
        # )
        written = df.write.jdbc(url=url, table=table, properties=properties,mode='append')
        data=[]
        counter=0

        
    counter+=1

In [14]:
# df = spark.readStream.format('kafka') \
#     .option('kafka.bootstrap.servers', 'localhost:9092') \
#     .option('subscribe', 'streamData') \
#     .load()



In [ ]:
# writer = spark.writeStream.format('console').outMode('append').trigger(timeProcessing='5 seconds')
# query=writer.start()

In [ ]:
# query.stop()